In [1]:
# Let's start learning
import torch
from torch_geometric.datasets import Planetoid
from cora_loader import CitationNetwork,CocitationNetwork,ConfigurationModelCitationNetwork
from torch.utils.data import DataLoader
from torch_geometric.nn import GCNConv,SAGEConv,GATConv,GraphConv,SGConv


from torch_geometric_node2vec import Node2Vec

# from models import MonoModel,BiModel,TriModel
from multi_layered_model import MonoModel,BiModel,TriModel
import numpy as np

In [6]:
import time
import torch.nn.functional as F

def run_and_eval_model(dataset,channels,modelType,architecture=MonoModel,epochs=200):
    # training process (without batches/transforms)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = architecture(modelType,dataset,channels).to(device)
#     print(model)
    data = dataset[0].to(device)
    optimizer = torch.optim.Adam(model.parameters(),lr=0.01,weight_decay=5e-4)
    model.train() # to enter training phase
    for epoch in range(epochs):
        optimizer.zero_grad() # saw this a lot in the beginning, maybe resetting gradients (not to accumulate)
        out = model(data) # this calls the forward method apparently
        loss = F.nll_loss(out[data.train_mask],data.y[data.train_mask]) # nice indexing, easy and short
        loss.backward() # magic: real back propagation step, takes care of the gradients and stuff
        optimizer.step() # maybe updates the params to be optimized
    model.eval() # enter eval phase
    _,pred = model(data).max(dim=1) # take prediction out of softmax
    correct = float(pred[data.test_mask].eq(data.y[data.test_mask]).sum().item())
    acc = correct / data.test_mask.sum().item()
    return acc

def eval_multiple(dataset,channels,modelType,architecture=MonoModel,runs=100,epochs=50):
    start = time.time()
    accs = []
    for i in range(runs):
        accs.append(run_and_eval_model(dataset,channels,modelType,architecture,epochs))
    elapsed_time = time.time() - start
    print('Elaplsed {}'.format(time.strftime("%H:%M:%S", time.gmtime(elapsed_time))))
    return sum(accs)/len(accs),np.std(np.array(accs)),accs

In [4]:
cocit = CitationNetwork('/tmp/CoCit0','CoCit',directed=False)

In [5]:
print('cocit accuracy: {:.4f}'.format(
    eval_multiple(cocit,[128],GCNConv,runs=1,epochs=200)[0]))
print('cocit accuracy: {:.4f}'.format(
    eval_multiple(cocit,[64],GCNConv,architecture=BiModel,runs=1,epochs=200)[0]))
print('cocit accuracy: {:.4f}'.format(
    eval_multiple(cocit,[42],GCNConv,architecture=TriModel,runs=1,epochs=200)[0]))

Elaplsed 00:00:41
cocit accuracy: 0.0900
Elaplsed 00:00:40
cocit accuracy: 0.0900
Elaplsed 00:01:19
cocit accuracy: 0.0720


In [3]:
cora = CitationNetwork('/tmp/cora','cora',directed=False)

In [4]:
citeseer = CitationNetwork('/tmp/citeseer','citeseer',directed=False)

In [5]:
PubMed = CitationNetwork('/tmp/PubMed','PubMed',directed=False)

In [12]:
coraDirected = CitationNetwork('/tmp/coraDirected','cora',directed=True)
coraReversed = CitationNetwork('/tmp/coraReversed','cora',directed=True,reverse=True)
citeseerDirected = CitationNetwork('/tmp/citeseerDirected','citeseer',directed=True)
citeseerReversed = CitationNetwork('/tmp/citeseerReversed','citeseer',directed=True,reverse=True)

## Accuracy on normal network

In [18]:
print('cora accuracy: {:.4f}'.format(
    eval_multiple(cora,[32],GCNConv,runs=100,epochs=200)[0]))
print('cora accuracy: {:.4f}'.format(
    eval_multiple(cora,[16],GCNConv,architecture=BiModel,runs=100,epochs=200)[0]))
print('cora accuracy: {:.4f}'.format(
    eval_multiple(cora,[12],GCNConv,architecture=TriModel,runs=100,epochs=200)[0]))

Elaplsed 00:01:19
cora accuracy: 0.7991
Elaplsed 00:01:51
cora accuracy: 0.7968
Elaplsed 00:02:19
cora accuracy: 0.8004


In [20]:
print('cora accuracy: {:.4f}'.format(
    eval_multiple(cora,[32],GATConv,runs=100,epochs=200)[0]))
print('cora accuracy: {:.4f}'.format(
    eval_multiple(cora,[16],GATConv,architecture=BiModel,runs=100,epochs=200)[0]))
print('cora accuracy: {:.4f}'.format(
    eval_multiple(cora,[12],GATConv,architecture=TriModel,runs=100,epochs=200)[0]))

Elaplsed 00:02:13
cora accuracy: 0.7593
Elaplsed 00:03:20
cora accuracy: 0.7756
Elaplsed 00:04:10
cora accuracy: 0.7830


In [13]:
print('cora accuracy: {:.4f}'.format(
    eval_multiple(cora,[128],GCNConv,runs=100,epochs=200)[0]))
print('cora accuracy: {:.4f}'.format(
    eval_multiple(cora,[64],GCNConv,architecture=BiModel,runs=100,epochs=200)[0]))
print('cora accuracy: {:.4f}'.format(
    eval_multiple(cora,[42],GCNConv,architecture=TriModel,runs=100,epochs=200)[0]))

Elaplsed 00:01:30
cora accuracy: 0.8016
Elaplsed 00:02:02
cora accuracy: 0.8072
Elaplsed 00:02:37
cora accuracy: 0.8067


In [14]:
print('cora accuracy: {:.4f}'.format(
    eval_multiple(cora,[128],GATConv,runs=100,epochs=200)[0]))
print('cora accuracy: {:.4f}'.format(
    eval_multiple(cora,[64],GATConv,architecture=BiModel,runs=100,epochs=200)[0]))
print('cora accuracy: {:.4f}'.format(
    eval_multiple(cora,[42],GATConv,architecture=TriModel,runs=100,epochs=200)[0]))

Elaplsed 00:02:22
cora accuracy: 0.7430
Elaplsed 00:03:22
cora accuracy: 0.7675
Elaplsed 00:04:13
cora accuracy: 0.7763


In [15]:
print('citeseer accuracy: {:.4f}'.format(
    eval_multiple(citeseer,[128],GCNConv,runs=100,epochs=200)[0]))
print('citeseer accuracy: {:.4f}'.format(
    eval_multiple(citeseer,[64],GCNConv,architecture=BiModel,runs=100,epochs=200)[0]))
print('citeseer accuracy: {:.4f}'.format(
    eval_multiple(citeseer,[42],GCNConv,architecture=TriModel,runs=100,epochs=200)[0]))

Elaplsed 00:01:32
citeseer accuracy: 0.7224
Elaplsed 00:02:09
citeseer accuracy: 0.7195
Elaplsed 00:02:35
citeseer accuracy: 0.7193


In [25]:
print('citeseer accuracy: {:.4f}'.format(
    eval_multiple(citeseer,[128],GATConvConv,runs=100,epochs=200)[0]))
print('citeseer accuracy: {:.4f}'.format(
    eval_multiple(citeseer,[64],GATConv,architecture=BiModel,runs=100,epochs=200)[0]))
print('citeseer accuracy: {:.4f}'.format(
    eval_multiple(citeseer,[42],GATConv,architecture=TriModel,runs=100,epochs=200)[0]))

Elaplsed 00:02:17
citeseer accuracy: 0.6981
Elaplsed 00:03:14
citeseer accuracy: 0.7195
Elaplsed 00:04:14
citeseer accuracy: 0.7232


In [19]:
print('cora accuracy: {:.4f}'.format(
    eval_multiple(cora,[128,128],GCNConv,runs=100,epochs=500)[0]))

Elaplsed 00:05:07
cora accuracy: 0.7867


In [20]:
print('cora accuracy: {:.4f}'.format(
    eval_multiple(cora,[64,64],GCNConv,architecture=BiModel,runs=100,epochs=500)[0]))

Elaplsed 00:07:22
cora accuracy: 0.7862


In [22]:
print('cora accuracy: {:.4f}'.format(
    eval_multiple(cora,[42,42],GCNConv,architecture=TriModel,runs=100,epochs=500)[0]))

Elaplsed 00:10:10
cora accuracy: 0.7837


In [28]:
print('cora accuracy: {:.4f}'.format(
    eval_multiple(cora,[128,64,32,16],GCNConv,runs=100,epochs=200)[0]))

Elaplsed 00:03:07
cora accuracy: 0.7736


In [29]:
print('cora accuracy: {:.4f}'.format(
    eval_multiple(cora,[64,32,16,8],GCNConv,architecture=BiModel,runs=100,epochs=200)[0]))

Elaplsed 00:05:14
cora accuracy: 0.7648


In [ ]:
print('cora accuracy: {:.4f}'.format(
    eval_multiple(cora,[42,21,10,5],GCNConv,architecture=BiModel,runs=100,epochs=200)[0]))

## Accuracy on configuration model

### Directed Case

In [11]:
coraConfDirected = ConfigurationModelCitationNetwork('/tmp/coraConfDirected','cora',directed=True)
citeseerConfDirected = ConfigurationModelCitationNetwork('/tmp/citeseerConfDirected','citeseer',directed=True)
PubMedConfDirected = ConfigurationModelCitationNetwork('/tmp/PubMedConfDirected','PubMed',directed=True)

In [13]:
meanAcc,stdAcc,acc = eval_multiple(coraConfDirected,[128],GCNConv,runs=100,epochs=200)
print('cora accuracy: {:.4f}+-({:.4f})'.format(
    meanAcc,stdAcc))

Elaplsed 00:01:27
cora accuracy: 0.2449+-(0.0153)


In [15]:
meanAcc,stdAcc,acc = eval_multiple(citeseerConfDirected,[128],GCNConv,runs=100,epochs=200)
print('citeseer accuracy: {:.4f}+-({:.4f})'.format(
    meanAcc,stdAcc))

Elaplsed 00:01:36
citeseer accuracy: 0.3261+-(0.0162)


In [16]:
meanAcc,stdAcc,acc = eval_multiple(PubMedConfDirected,[128],GCNConv,runs=100,epochs=200)
print('PubMed accuracy: {:.4f}+-({:.4f})'.format(
    meanAcc,stdAcc))

Elaplsed 00:01:50
PubMed accuracy: 0.4953+-(0.0165)


In [12]:
meanAcc,stdAcc,acc = eval_multiple(coraConfDirected,[128],SAGEConv,runs=100,epochs=200)
print('cora accuracy: {:.4f}+-({:.4f})'.format(
    meanAcc,stdAcc))
meanAcc,stdAcc,acc = eval_multiple(citeseerConfDirected,[128],SAGEConv,runs=100,epochs=200)
print('citeseer accuracy: {:.4f}+-({:.4f})'.format(
    meanAcc,stdAcc))
meanAcc,stdAcc,acc = eval_multiple(PubMedConfDirected,[128],SAGEConv,runs=100,epochs=200)
print('PubMed accuracy: {:.4f}+-({:.4f})'.format(
    meanAcc,stdAcc))

Elaplsed 00:01:14
cora accuracy: 0.1729+-(0.0097)
Elaplsed 00:01:32
citeseer accuracy: 0.2572+-(0.0098)
Elaplsed 00:02:20
PubMed accuracy: 0.3578+-(0.0239)


### Reversed Case

In [4]:
coraConfReversed = ConfigurationModelCitationNetwork('/tmp/coraConfReversed','cora',directed=True,reverse=True)
citeseerConfReversed = ConfigurationModelCitationNetwork('/tmp/citeseerConfReversed','citeseer',directed=True,reverse=True)
PubMedConfReversed = ConfigurationModelCitationNetwork('/tmp/PubMedConfReversed','PubMed',directed=True,reverse=True)

In [7]:
meanAcc,stdAcc,acc = eval_multiple(coraConfReversed,[128],GCNConv,runs=100,epochs=200)
print('cora accuracy: {:.4f}+-({:.4f})'.format(
    meanAcc,stdAcc))

Elaplsed 00:01:34
cora accuracy: 0.3067+-(0.0127)


In [8]:
meanAcc,stdAcc,acc = eval_multiple(citeseerConfReversed,[128],GCNConv,runs=100,epochs=200)
print('citeseer accuracy: {:.4f}+-({:.4f})'.format(
    meanAcc,stdAcc))

Elaplsed 00:01:47
citeseer accuracy: 0.3278+-(0.0142)


In [9]:
meanAcc,stdAcc,acc = eval_multiple(PubMedConfReversed,[128],GCNConv,runs=100,epochs=200)
print('PubMed accuracy: {:.4f}+-({:.4f})'.format(
    meanAcc,stdAcc))

Elaplsed 00:02:10
PubMed accuracy: 0.5947+-(0.0056)


In [10]:
meanAcc,stdAcc,acc = eval_multiple(coraConfReversed,[128],SAGEConv,runs=100,epochs=200)
print('cora accuracy: {:.4f}+-({:.4f})'.format(
    meanAcc,stdAcc))
meanAcc,stdAcc,acc = eval_multiple(citeseerConfReversed,[128],SAGEConv,runs=100,epochs=200)
print('citeseer accuracy: {:.4f}+-({:.4f})'.format(
    meanAcc,stdAcc))
meanAcc,stdAcc,acc = eval_multiple(PubMedConfReversed,[128],SAGEConv,runs=100,epochs=200)
print('PubMed accuracy: {:.4f}+-({:.4f})'.format(
    meanAcc,stdAcc))

Elaplsed 00:01:17
cora accuracy: 0.2274+-(0.0116)
Elaplsed 00:01:31
citeseer accuracy: 0.2688+-(0.0143)
Elaplsed 00:02:07
PubMed accuracy: 0.5722+-(0.0161)


### Undirected Case

In [6]:
coraConf = ConfigurationModelCitationNetwork('/tmp/coraConf','cora',directed=False)
citeseerConf = ConfigurationModelCitationNetwork('/tmp/citeseerConf','citeseer',directed=False)
PubMedConf = ConfigurationModelCitationNetwork('/tmp/PubMedConf','PubMed',directed=False)

In [10]:
print('cora accuracy: {:.4f}'.format(
    eval_multiple(coraConf,[128],GCNConv,runs=100,epochs=200)[0]))
print('cora accuracy: {:.4f}'.format(
    eval_multiple(coraConf,[64],GCNConv,architecture=BiModel,runs=100,epochs=200)[0]))
print('cora accuracy: {:.4f}'.format(
    eval_multiple(coraConf,[42],GCNConv,architecture=TriModel,runs=100,epochs=200)[0]))

Elaplsed 00:01:20
cora accuracy: 0.3145
Elaplsed 00:01:51
cora accuracy: 0.3079
Elaplsed 00:02:29
cora accuracy: 0.3088


In [9]:
print('citeseer accuracy: {:.4f}'.format(
    eval_multiple(citeseerConf,[128],GCNConv,runs=100,epochs=200)[0]))
print('citeseer accuracy: {:.4f}'.format(
    eval_multiple(citeseerConf,[64],GCNConv,architecture=BiModel,runs=100,epochs=200)[0]))
print('citeseer accuracy: {:.4f}'.format(
    eval_multiple(citeseerConf,[42],GCNConv,architecture=TriModel,runs=100,epochs=200)[0]))

Elaplsed 00:01:33
citeseer accuracy: 0.3533
Elaplsed 00:02:05
citeseer accuracy: 0.3401
Elaplsed 00:02:40
citeseer accuracy: 0.3423


In [4]:
print('PubMed accuracy: {:.4f}'.format(
    eval_multiple(PubMedConf,[128],GCNConv,runs=100,epochs=200)[0]))
print('PubMed accuracy: {:.4f}'.format(
    eval_multiple(PubMedConf,[64],GCNConv,architecture=BiModel,runs=100,epochs=200)[0]))
print('PubMed accuracy: {:.4f}'.format(
    eval_multiple(PubMedConf,[42],GCNConv,architecture=TriModel,runs=100,epochs=200)[0]))

Elaplsed 00:05:39
PubMed accuracy: 0.5221
Elaplsed 00:08:18
PubMed accuracy: 0.5223
Elaplsed 00:11:27
PubMed accuracy: 0.5228


In [9]:
print('cora accuracy: {:.4f}'.format(
    eval_multiple(coraConf,[128],SAGEConv,runs=100,epochs=200)[0]))
print('cora accuracy: {:.4f}'.format(
    eval_multiple(coraConf,[64],SAGEConv,architecture=BiModel,runs=100,epochs=200)[0]))
print('cora accuracy: {:.4f}'.format(
    eval_multiple(coraConf,[42],SAGEConv,architecture=TriModel,runs=100,epochs=200)[0]))

Elaplsed 00:01:18
cora accuracy: 0.2237
Elaplsed 00:01:56
cora accuracy: 0.2172
Elaplsed 00:02:15
cora accuracy: 0.2206


In [10]:
print('citeseer accuracy: {:.4f}'.format(
    eval_multiple(citeseerConf,[128],SAGEConv,runs=100,epochs=200)[0]))
print('citeseer accuracy: {:.4f}'.format(
    eval_multiple(citeseerConf,[64],SAGEConv,architecture=BiModel,runs=100,epochs=200)[0]))
print('citeseer accuracy: {:.4f}'.format(
    eval_multiple(citeseerConf,[42],SAGEConv,architecture=TriModel,runs=100,epochs=200)[0]))

Elaplsed 00:01:44
citeseer accuracy: 0.2866
Elaplsed 00:02:03
citeseer accuracy: 0.2605
Elaplsed 00:02:35
citeseer accuracy: 0.2736


In [11]:
print('PubMed accuracy: {:.4f}'.format(
    eval_multiple(PubMedConf,[128],SAGEConv,runs=100,epochs=200)[0]))
print('PubMed accuracy: {:.4f}'.format(
    eval_multiple(PubMedConf,[64],SAGEConv,architecture=BiModel,runs=100,epochs=200)[0]))
print('PubMed accuracy: {:.4f}'.format(
    eval_multiple(PubMedConf,[42],SAGEConv,architecture=TriModel,runs=100,epochs=200)[0]))

Elaplsed 00:02:41
PubMed accuracy: 0.4473
Elaplsed 00:02:46
PubMed accuracy: 0.4507
Elaplsed 00:03:03
PubMed accuracy: 0.4414


## BibCo

In [4]:
bib_cora = CocitationNetwork('/tmp/bib_cora','cora','bibcoupling')

In [5]:
coc_cora = CocitationNetwork('/tmp/coc_cora','cora','cocitation')

In [6]:
coc_citeseer = CocitationNetwork('/tmp/coc_citeseer','citeseer','cocitation')
bib_citeseer = CocitationNetwork('/tmp/bib_citeseer','citeseer','bibcoupling')

In [4]:
def eval_all(conv,data,runs = 100,epochs=50):
    monogcn,monogcns = eval_multiple(data,128,conv,runs=runs,epochs=epochs)
    print('Mono accuracy: {:.4f}'.format(monogcn))
    bigcn,bigcns = eval_multiple_bi(data,64,64,conv,runs=runs,epochs=epochs)
    print('Bi   accuracy: {:.4f}'.format(bigcn))
    trigcn,trigcns = eval_multiple_tri(data,64,32,32,conv,runs=runs,epochs=epochs)
    print('Tri  accuracy: {:.4f}'.format(trigcn))

In [5]:
print('cora accuracy: {:.4f}'.format(
    eval_multiple(cora,128,GCNConv,runs=100,epochs=200)[0]))

Elaplsed 00:01:33
cora accuracy: 0.8013


In [7]:
print('cora accuracy: {:.4f}'.format(
    eval_multiple(cora,128,SAGEConv,runs=100,epochs=200)[0]))

Elaplsed 00:01:13
cora accuracy: 0.7640


In [ ]:
print('cora accuracy: {:.4f}'.format(
    eval_multiple(cora,128,GCNConv,runs=100,epochs=200)[0]))

## Directed

In [13]:
print('coraDirected accuracy: {:.4f}'.format(
    eval_multiple(coraDirected,128,GCNConv,runs=100,epochs=200)[0]))

Elaplsed 00:01:42
coraDirected accuracy: 0.5709


In [14]:
print('citeseerDirected accuracy: {:.4f}'.format(
    eval_multiple(citeseerDirected,128,GCNConv,runs=100,epochs=200)[0]))

Elaplsed 00:02:04
citeseerDirected accuracy: 0.5881


## Reversed

In [15]:
print('coraReversed accuracy: {:.4f}'.format(
    eval_multiple(coraReversed,128,GCNConv,runs=100,epochs=200)[0]))

Elaplsed 00:05:32
coraReversed accuracy: 0.5970


In [16]:
print('citeseerReversed accuracy: {:.4f}'.format(
    eval_multiple(citeseerReversed,128,GCNConv,runs=100,epochs=200)[0]))

Elaplsed 00:01:39
citeseerReversed accuracy: 0.5547


## Undirected

In [15]:
eval_all(GCNConv,cora,100,200)

Elaplsed 00:01:17
Mono accuracy: 0.8011
Elaplsed 00:01:50
Bi   accuracy: 0.8071
Elaplsed 00:02:19
Tri  accuracy: 0.8061


In [16]:
eval_all(GCNConv,citeseer,100,200)

Elaplsed 00:01:31
Mono accuracy: 0.7223
Elaplsed 00:02:02
Bi   accuracy: 0.7184
Elaplsed 00:02:25
Tri  accuracy: 0.7201


## Cocitation

In [10]:
eval_all(GCNConv,coc_cora,10,200)

Elaplsed 00:00:10
Mono accuracy: 0.7045
Elaplsed 00:00:11
Bi   accuracy: 0.7111
Elaplsed 00:00:14
Tri  accuracy: 0.7072


In [13]:
eval_all(GCNConv,coc_citeseer,10,200)

Elaplsed 00:00:10
Mono accuracy: 0.6969
Elaplsed 00:00:11
Bi   accuracy: 0.6998
Elaplsed 00:00:14
Tri  accuracy: 0.7015


## Bibliographic coupling

In [11]:
eval_all(GCNConv,bib_cora,10,200)

Elaplsed 00:00:08
Mono accuracy: 0.6882
Elaplsed 00:00:10
Bi   accuracy: 0.6809
Elaplsed 00:00:13
Tri  accuracy: 0.6887


In [14]:
eval_all(GCNConv,bib_citeseer,10,200)

Elaplsed 00:00:09
Mono accuracy: 0.6721
Elaplsed 00:00:11
Bi   accuracy: 0.6674
Elaplsed 00:00:14
Tri  accuracy: 0.6664


## More epochs

In [18]:
eval_all(GCNConv,cora,100,500)

Elaplsed 00:03:21
Mono accuracy: 0.7991
Elaplsed 00:04:34
Bi   accuracy: 0.8029
Elaplsed 00:05:44
Tri  accuracy: 0.8033


In [19]:
eval_all(GCNConv,citeseer,100,500)

Elaplsed 00:03:46
Mono accuracy: 0.7203
Elaplsed 00:04:57
Bi   accuracy: 0.7163
Elaplsed 00:05:57
Tri  accuracy: 0.7162


## Other models

In [28]:
eval_all(SAGEConv,100)

Elaplsed 00:00:17
Mono accuracy: 0.7814
Elaplsed 00:00:24
Bi   accuracy: 0.7728
Elaplsed 00:00:30
Tri  accuracy: 0.7774


In [33]:
eval_all(GATConv,100)

Elaplsed 00:00:56
Mono accuracy: 0.7532
Elaplsed 00:01:23
Bi   accuracy: 0.7680
Elaplsed 00:01:47
Tri  accuracy: 0.7707


In [30]:
eval_all(GraphConv,100)

Elaplsed 00:00:13
Mono accuracy: 0.7496
Elaplsed 00:00:20
Bi   accuracy: 0.7304
Elaplsed 00:00:34
Tri  accuracy: 0.7425


In [32]:
eval_all(SGConv,100)

Elaplsed 00:00:32
Mono accuracy: 0.7960
Elaplsed 00:00:47
Bi   accuracy: 0.7823
Elaplsed 00:00:58
Tri  accuracy: 0.7865


In [40]:
monogcn,monogcns = eval_multiple(coraUndirected,128,GCNConv,runs=100)
print('Mono accuracy: {:.4f}'.format(monogcn))
trigcn,trigcns = eval_multiple_tri(coraUndirected,128,8,8,GCNConv,runs=100)
print('Tri  accuracy: {:.4f}'.format(trigcn))
trigcn,trigcns = eval_multiple_tri(coraUndirected,128,1,1,GCNConv,runs=100)
print('Tri  accuracy: {:.4f}'.format(trigcn))

Elaplsed 00:00:34
Mono accuracy: 0.7978
Elaplsed 00:00:59
Tri  accuracy: 0.7970
Elaplsed 00:00:58
Tri  accuracy: 0.7973


In [42]:
trigcn,trigcns = eval_multiple_tri(coraUndirected,112,8,8,GCNConv,runs=100)
print('Tri  accuracy: {:.4f}'.format(trigcn))

Elaplsed 00:00:59
Tri  accuracy: 0.7978


In [43]:
trigcn,trigcns = eval_multiple_tri(coraUndirected,64,32,32,GCNConv,runs=100)
print('Tri  accuracy: {:.4f}'.format(trigcn))

Elaplsed 00:00:59
Tri  accuracy: 0.7977
